# Converting PCD to 3D Mesh
The pipeline function has uses of Open3d's mesh creating methods but decent outputs have not been achieved.

**Table of Contents** -- objects in red have not been well-tested
1. [Example pipeline function](#pipe)
   - Voxel Downsampling
   - Outlier removal
       - statistical removal
       - radius removal
   - Normal estimation
   - <a style="color:red;">Mesh creation</a>
       - alpha shapes
       - ball pivoting
       - poisson surface
   - <a style="color:red;">Smoothing</a>
   - Visualization
3. [Use this cell to edit bounds and define objects](#defineobj)
4. [Run this cell to create an object file](#saveobj)
5. [Functions for 1, 2, and 3 color palettes](#colors)
6. [Assigning object colors](#objcolors)
7. [Writing object to a scene](#scene)
8. [Example of loading and visualizing colorized scene](#viz)
9. [Example images](#images)

## Example pipeline function<a name="pipe"></a>

In [5]:
import open3d as o3d
import open3d.core as o3c
import pandas as pd
import numpy as np
import time


def pipeline(xyz, file_name=False,
             voxel_down=False, # voxel down
             remove_statistical_outliers=False, neighbors=20, std=2.0, display_inliers_outliers=False, # stat outlier
             remove_radius_outliers=False, nb_points=16, radius=0.05, # radius outlier
             estimate_normals=False, # normals
             alpha_shapes=False, alpha=0.5, # alpha surface
             ball_pivoting=False, radii=[0.005, 0.01, 0.02, 0.04], # ball pivoting
             poisson_surface=False, poisson_depth=9, # poisson surface
             smoothing=False, smoothing_iters=5, smoothing_lambda=0.5, smoothing_mu=0.5, # smoothing
             viz=False): # visualization on/off

    def change_background_to_black(vis):
        opt = vis.get_render_option()
        opt.background_color = np.asarray([0, 0, 0])
        return False

    key_to_callback = {}
    key_to_callback[ord("K")] = change_background_to_black
    
    # if no file name specified
    if not file_name:
        file_name = "Objects/PCD_visualization_" + str(time.time()).replace('.','_') + ".ply"
        
    # 1. Data conversion
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    # white_color = [255, 255, 255, 0.5]
    # colors = np.tile(white_color, (len(pcd.points), 1))
    # for i in range(len(pcd.colors)):
    #     pcd.colors[i] = white_color
    # ----------------------------------------------------------------------
    # 2. Voxel downsampling
    if voxel_down == True:
        pcd = pcd.voxel_down_sample(voxel_size=0.02)
        # write to file:
        o3d.io.write_point_cloud(file_name, pcd)
        # must read pcd file back through o3d.t
    # ----------------------------------------------------------------------
    # 3. Outlier removal
    if remove_statistical_outliers == True:
        pcd = o3d.io.read_point_cloud(file_name)  
        # must be read from o3d.t.io.read_point_cloud("file_name_here")
        cl, ind = pcd.remove_statistical_outlier(nb_neighbors=neighbors, std_ratio=std)
        o3d.io.write_point_cloud(file_name, pcd)

        if display_inliers_outliers == True:
            # function to display detected outliers in red
            def display_inlier_outlier(cloud, ind):
                inlier_cloud = cloud.select_by_index(ind)
                outlier_cloud = cloud.select_by_index(ind, invert=True)
                outlier_cloud = outlier_cloud.paint_uniform_color([1.0, 0, 0])
                inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
                inlier_cloud = o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])
            pcd = o3d.io.read_point_cloud(file_name)    
            display_inlier_outlier(pcd, ind)

    if remove_radius_outliers == True:
        pcd = o3d.io.read_point_cloud(file_name)  
        # must be read from o3d.t.io.read_point_cloud("file_name_here")
        cl, ind = pcd.remove_radius_outlier(nb_points=16, radius=0.05)
        o3d.io.write_point_cloud(file_name, pcd)

        if display_inliers_outliers == True:
            # function to display detected outliers in red
            def display_inlier_outlier(cloud, ind):
                inlier_cloud = cloud.select_by_index(ind)
                outlier_cloud = cloud.select_by_index(ind, invert=True)
                outlier_cloud = outlier_cloud.paint_uniform_color([1.0, 0, 0])
                inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
                inlier_cloud = o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])
            pcd = o3d.io.read_point_cloud(file_name)    
            display_inlier_outlier(pcd, ind)
    # ----------------------------------------------------------------------
    # 4. Set bounds
    # if set_bounds == True:
    #     pcd = o3d.io.read_point_cloud(file_name)  
    #     points = np.asarray(pcd.points)
    #     x_min, x_max = X[0], X[1]
    #     y_min, y_max = Y[0], Y[1]
    #     z_min, z_max = Z[0], Z[1]
    #     filtered_indices = np.where(
    #     (points[:, 0] >= x_min) & (points[:, 0] <= x_max) &
    #     (points[:, 1] >= y_min) & (points[:, 1] <= y_max) &
    #     (points[:, 2] >= z_min) & (points[:, 2] <= z_max))
    #     pcd = pcd.select_by_index(filtered_indices)
    #     o3d.io.write_point_cloud(file_name, pcd)
    # ----------------------------------------------------------------------
    # 5. Non-permanent object removal
    # ----------------------------------------------------------------------
    # 6. Triangulation of PCD
    # ----------------------------------------------------------------------
    # 7. Vertex normal estimation
    if estimate_normals == True:
        pcd = o3d.io.read_point_cloud(file_name)    
        pcd.estimate_normals(
            search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
        o3d.io.write_point_cloud(file_name, pcd)
    # ----------------------------------------------------------------------        
    # 8. Surface mesh creation
    # 8.1 Alpha shapes
    if (alpha_shapes == True) and (ball_pivoting == False) and (poisson_surface == False): 
        pcd = o3d.io.read_point_cloud(file_name)    
        tetra_mesh, pt_map = o3d.geometry.TetraMesh.create_from_point_cloud(pcd)
        mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(
            pcd, alpha, tetra_mesh, pt_map)
        mesh.compute_vertex_normals()
        file_name = "triangle_mesh_" + file_name
        o3d.io.write_triangle_mesh(file_name, mesh)
    #  - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    # 8.2 Ball pivoting
    elif (ball_pivoting == True) and (alpha_shapes == False) and (poisson_surface == False):
        pcd = o3d.io.read_point_cloud(file_name)    
        mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
        pcd, o3d.utility.DoubleVector(radii))
        file_name = "triangle_mesh_" + file_name
        o3d.io.write_triangle_mesh(file_name, mesh)
    #  - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -   
    # 8.3 Poisson Surface reconstruction
    elif (poisson_surface == True) and (ball_pivoting == False) and (alpha_shapes == False):
        pcd = o3d.io.read_point_cloud(file_name)    
        mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=poisson_depth)
        o3d.io.write_triangle_mesh(file_name, mesh)
    # ----------------------------------------------------------------------
    # 9. Smoothing
    if smoothing: 
        mesh = mesh.filter_smooth_taubin(number_of_iterations=smoothing_iters, lambda_filter=smoothing_lambda, mu=smoothing_mu)
        o3d.io.write_triangle_mesh(file_name, mesh)
    # ----------------------------------------------------------------------
    # 10. Colorization
    # ----------------------------------------------------------------------
    # 11. Visualization
    if viz == True:
        R = 0
        G = 0
        B = 0
        print(R, G, B)
        try:
            aabb = pcd.get_axis_aligned_bounding_box()
            aabb.color = (R, G, B)
            mesh.paint_uniform_color([R, G, B])
            o3d.visualization.draw_geometries_with_key_callbacks([mesh, aabb], key_to_callback)
            # o3d.visualization.draw([mesh, aabb])
        except:
            aabb = pcd.get_axis_aligned_bounding_box()
            aabb.color = (R, G, B)
            pcd.paint_uniform_color([R, G, B])
            o3d.visualization.draw_geometries_with_key_callbacks([pcd, aabb], key_to_callback)
            # o3d.visualization.draw([pcd, aabb])
            
    return file_name

## Use this cell to edit bounds and define objects <a name="defineobj"></a>

In [6]:
# .pcap to .csv conversion
# ouster-cli source <PCAP> convert <out.csv>

import pandas as pd

file_path = "Data/clean_monroe.csv"
df = pd.read_csv(file_path)

set_bounds = True
linear_bounds = False

x_min =  -50
x_max =  50
y_min =  -100
y_max =  100
z_min = -10
z_max = 50

# (-8, 13), (50, 2)
m = 17/(-58)
mi = -1*m**-1
b = 13-(88/58)

if set_bounds:
    df = df[df['X1 (m)'] >= x_min]
    df = df[df['X1 (m)'] <= x_max]
    df = df[df['Y1 (m)'] >= y_min]
    df = df[df['Y1 (m)'] <= y_max]
    df = df[df['Z1 (m)'] >= z_min]
    df = df[df['Z1 (m)'] <= z_max]

if linear_bounds:
    mask = df['Y1 (m)'] <= m * df['X1 (m)'] + b - 1
    df = df[mask]
    mask = df['Y1 (m)'] >= m * df['X1 (m)'] + b - 2.3
    df = df[mask]
    # mask = df['Y1 (m)'] >= mi * df['X1 (m)'] - 11
    # df = df[mask]
    # mask = df['Y1 (m)'] <= mi * df['X1 (m)'] + 21
    # df = df[mask]

x = np.array(df['X1 (m)'][1:])
y = np.array(df['Y1 (m)'][1:])
z = np.array(df['Z1 (m)'][1:])
xyz = np.array([x,y,z]).T
# radii_measures = np.linspace(start=.5, stop=2, num=5)

## Run this cell to create an object file <a name="saveobj"></a>

In [ ]:
file = "Objects/monroe_example.ply"

pipeline(xyz,
         file_name=file,
         voxel_down=True, 
         remove_statistical_outliers=False, neighbors=10, std=2.0,
         estimate_normals=False,
         viz=True)

# pcd = o3d.io.read_point_cloud(file)  
# o3d.visualization.draw_geometries([pcd])

## Functions for 1, 2, and 3 color palettes <a name="colors"></a>

In [7]:
import random 

# Creates random range of 3 defined colors
def three_color_palette(pcd, c1_min, c1_max, c2_min, c2_max, c3_min, c3_max):
    # Create a list of colors
    colors = []
    for _ in range(len(pcd.points)):
        color_range = random.randint(0, 2)  # Randomly select 0, 1, or 2
        if color_range == 0:
            color = [random.randint(c1_min[0], c1_max[0]),
                     random.randint(c1_min[1], c1_max[1]),
                     random.randint(c1_min[2], c1_max[2])]
        elif color_range == 1:
            color = [random.randint(c2_min[0], c2_max[0]),
                     random.randint(c2_min[1], c2_max[1]),
                     random.randint(c2_min[2], c2_max[2])]
        else:
            color = [random.randint(c3_min[0], c3_max[0]),
                     random.randint(c3_min[1], c3_max[1]),
                     random.randint(c3_min[2], c3_max[2])]
        colors.append(color)

    return colors

# Creates random range of 2 defined colors
def two_color_palette(pcd, c1_min, c1_max, c2_min, c2_max):
    colors = []
    for _ in range(len(pcd.points)):
        color_range = random.randint(0, 1)  # Randomly select 0, 1
        if color_range == 0:
            color = [random.randint(c1_min[0], c1_max[0]),
                     random.randint(c1_min[1], c1_max[1]),
                     random.randint(c1_min[2], c1_max[2])]
        else:
            color = [random.randint(c2_min[0], c2_max[0]),
                     random.randint(c2_min[1], c2_max[1]),
                     random.randint(c2_min[2], c2_max[2])]
        colors.append(color)

    return colors

# Creates random range of 3 defined colors
def one_color_palette(pcd, c1_min, c1_max):
    # Create a list of colors
    colors = []
    for _ in range(len(pcd.points)):
        color = [random.randint(c1_min[0], c1_max[0]),
                 random.randint(c1_min[1], c1_max[1]),
                 random.randint(c1_min[2], c1_max[2])]
        colors.append(color)

    return colors

## Assigning object colors <a name="objcolors"></a>

In [17]:
# coloring individual objects

bush = o3d.io.read_point_cloud("Objects/front_bushes.ply")
c1_mn = [101, 133, 29]
c1_mx = [181, 208, 97]
c2_mn = [0, 87, 0]
c2_mx = [37, 135, 37]
colors = two_color_palette(bush, c1_mn, c1_mx, c2_mn, c2_mx)
bush.colors = o3d.utility.Vector3dVector(np.array(colors) / 255.0)
o3d.io.write_point_cloud("Objects/front_bushes.ply", bush)

bush2 =  o3d.io.read_point_cloud("Objects/other_bush.ply")
c1_mn = [101, 133, 29]
c1_mx = [181, 208, 97]
c2_mn = [0, 87, 0]
c2_mx = [37, 135, 37]
colors = two_color_palette(bush2, c1_mn, c1_mx, c2_mn, c2_mx)
bush2.colors = o3d.utility.Vector3dVector(np.array(colors) / 255.0)
o3d.io.write_point_cloud("Objects/other_bush.ply", bush2)

bush3 =  o3d.io.read_point_cloud("Objects/bush3.ply")
c1_mn = [101, 133, 29]
c1_mx = [181, 208, 97]
c2_mn = [0, 87, 0]
c2_mx = [37, 135, 37]
colors = two_color_palette(bush3, c1_mn, c1_mx, c2_mn, c2_mx)
bush3.colors = o3d.utility.Vector3dVector(np.array(colors) / 255.0)
o3d.io.write_point_cloud("Objects/bush3.ply", bush3)

bush4 =  o3d.io.read_point_cloud("Objects/bush4.ply")
c1_mn = [101, 133, 29]
c1_mx = [181, 208, 97]
c2_mn = [0, 87, 0]
c2_mx = [37, 135, 37]
colors = two_color_palette(bush4, c1_mn, c1_mx, c2_mn, c2_mx)
bush4.colors = o3d.utility.Vector3dVector(np.array(colors) / 255.0)
o3d.io.write_point_cloud("Objects/bush4.ply", bush4)

monroe_sign = o3d.io.read_point_cloud("Objects/monroe_sign.ply")
c1_mn = [36, 54, 94]
c1_mx = [39, 64, 107]
colors = one_color_palette(monroe_sign, c1_mn, c1_mx)
monroe_sign.colors = o3d.utility.Vector3dVector(np.array(colors) / 255.0)
o3d.io.write_point_cloud("Objects/monroe_sign.ply", monroe_sign)

front_face = o3d.io.read_point_cloud("Objects/front_face.ply")
c1_mn = [176, 180, 182]
c1_mx = [222, 227, 231]
colors = one_color_palette(front_face, c1_mn, c1_mx)
front_face.colors = o3d.utility.Vector3dVector(np.array(colors) / 255.0)
o3d.io.write_point_cloud("Objects/front_face.ply", front_face)

threshold = o3d.io.read_point_cloud("Objects/threshold.ply")
c1_mn = [138, 128, 126]
c1_mx = [203, 184, 167]
colors = one_color_palette(threshold, c1_mn, c1_mx)
threshold.colors = o3d.utility.Vector3dVector(np.array(colors) / 255.0)
o3d.io.write_point_cloud("Objects/threshold.ply", threshold)

building_label = o3d.io.read_point_cloud("Objects/building_label.ply")
c1_mn = [208, 238, 239]
c1_mx = [231, 240, 240]
colors = one_color_palette(building_label, c1_mn, c1_mx)
building_label.colors = o3d.utility.Vector3dVector(np.array(colors) / 255.0)
o3d.io.write_point_cloud("Objects/building_label.ply", building_label)

doors = o3d.io.read_point_cloud("Objects/doors.ply")
c1_mn = [75, 68, 64]
c1_mx = [87, 84, 69]
colors = one_color_palette(doors, c1_mn, c1_mx)
doors.colors = o3d.utility.Vector3dVector(np.array(colors) / 255.0)
o3d.io.write_point_cloud("Objects/doors.ply", doors)

building = o3d.io.read_point_cloud("Objects/building.ply")
c1_mn = [142, 105, 87]
c1_mx = [211, 139, 121]
colors = one_color_palette(building, c1_mn, c1_mx)
building.colors = o3d.utility.Vector3dVector(np.array(colors) / 255.0)
o3d.io.write_point_cloud("Objects/building.ply", building)

lightpost = o3d.io.read_point_cloud("Objects/light_post.ply")
c1_mx = [120, 111, 105]
c1_mn = [104, 88, 79]
colors = one_color_palette(lightpost, c1_mn, c1_mx)
lightpost.colors = o3d.utility.Vector3dVector(np.array(colors) / 255.0)
o3d.io.write_point_cloud("Objects/light_post.ply", lightpost)

tree = o3d.io.read_point_cloud("Objects/tree.ply")
yellow_min = [124, 206, 0]
yellow_max = [255, 222, 0]
orange_min = [255, 154, 0]
orange_max = [255, 196, 0]
red_min = [255, 111, 0]
red_max = [255, 154, 0]
colors = three_color_palette(tree, yellow_min, yellow_max, orange_min, orange_max, red_min, red_max)
tree.colors = o3d.utility.Vector3dVector(np.array(colors) / 255.0)
o3d.io.write_point_cloud("Objects/tree.ply", tree)

trunk = o3d.io.read_point_cloud("Objects/trunk.ply")
c1_mx = [132, 118, 107]
c1_mn = [67, 66, 46]
colors = one_color_palette(trunk, c1_mn, c1_mx)
trunk.colors = o3d.utility.Vector3dVector(np.array(colors) / 255.0)
o3d.io.write_point_cloud("Objects/trunk.ply", trunk)

grass = o3d.io.read_point_cloud("Objects/grass.ply")
c1_mx = [224, 209, 186]
c1_mn = [79, 116, 13]
c2_mx = [125, 135, 71]
c2_mn = [61, 67, 31]
c3_mx = [182, 172, 108]
c3_mn = [176, 171, 107]
colors = three_color_palette(grass, c1_mn, c1_mx, c2_mn, c2_mx, c3_mn, c3_mx)
grass.colors = o3d.utility.Vector3dVector(np.array(colors) / 255.0)
o3d.io.write_point_cloud("Objects/grass.ply", grass)

issa_head = o3d.io.read_point_cloud("Objects/issa_head.ply")
issa_head.paint_uniform_color([0.8784313, 0.772549019, 0.654901])
o3d.io.write_point_cloud("Objects/issa_head.ply", issa_head)

issa_shirt = o3d.io.read_point_cloud("Objects/issa_shirt.ply")
issa_shirt.paint_uniform_color([0.6980392, 0.1607843, 0.3098039])
o3d.io.write_point_cloud("Objects/issa_shirt.ply", issa_shirt)

issa_pants = o3d.io.read_point_cloud("Objects/issa_pants.ply")
issa_pants.paint_uniform_color([0.184313725, 0.282352, 0.3490196])
o3d.io.write_point_cloud("Objects/issa_pants.ply", issa_pants)

nicholas_head = o3d.io.read_point_cloud("Objects/nicholas_head.ply")
nicholas_head.paint_uniform_color([0.3568627, 0.30196078, 0.290196])
o3d.io.write_point_cloud("Objects/nicholas_head.ply", nicholas_head)

nicholas_shirt = o3d.io.read_point_cloud("Objects/nicholas_shirt.ply")
nicholas_shirt.paint_uniform_color([0.38823529, 0.8588235, 0.8392156])
o3d.io.write_point_cloud("Objects/nicholas_shirt.ply", nicholas_shirt)

planter = o3d.io.read_point_cloud("Objects/planter.ply")
planter.paint_uniform_color([0.803921, 0.807843, 0.815686])
o3d.io.write_point_cloud("Objects/planter.ply", planter)

sidewalk = o3d.io.read_point_cloud("Objects/sidewalk.ply")
c1_mn = [205, 203, 191]
c1_mx = [236, 233, 215]
colors = one_color_palette(sidewalk, c1_mn, c1_mx)
sidewalk.colors = o3d.utility.Vector3dVector(np.array(colors) / 255.0)
o3d.io.write_point_cloud("Objects/sidewalk.ply", sidewalk)

sidewalk2 = o3d.io.read_point_cloud("Objects/sidewalk2.ply")
c1_mn = [205, 203, 191]
c1_mx = [236, 233, 215]
colors = one_color_palette(sidewalk2, c1_mn, c1_mx)
sidewalk2.colors = o3d.utility.Vector3dVector(np.array(colors) / 255.0)
o3d.io.write_point_cloud("Objects/sidewalk2.ply", sidewalk2)

frontpaver = o3d.io.read_point_cloud("Objects/frontpaver.ply")
c1_mn = [205, 203, 191]
c1_mx = [236, 233, 215]
colors = one_color_palette(frontpaver, c1_mn, c1_mx)
frontpaver.colors = o3d.utility.Vector3dVector(np.array(colors) / 255.0)
o3d.io.write_point_cloud("Objects/frontpaver.ply", frontpaver)

def change_background_to_black(vis):
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0, 0, 0])
    return False

key_to_callback = {}
key_to_callback[ord("K")] = change_background_to_black


## Writing object to a scene <a name="scene"></a>

In [11]:
# object order matters: objects assigned first take color precendence
scene_objects = [issa_head, 
                 issa_shirt, 
                 issa_pants, 
                 nicholas_head, 
                 nicholas_shirt,
                 
                 monroe_sign, 
                 lightpost, 
                 
                 bush, 
                 bush2, 
                 bush3, 
                 bush4, 
                 trunk, 
                 tree,

                 sidewalk, 
                 sidewalk2, 
                 frontpaver, 
                 planter,
                 
                 grass,
                 building_label,
                 threshold, 
                 front_face, 
                 building, 
                 doors,]

final_colorized_scene = o3d.geometry.PointCloud()

for obj in scene_objects:
    final_colorized_scene += obj

o3d.io.write_point_cloud("monroe_final_scene.ply", final_colorized_scene)

True

## Example of loading and visualizing colorized scene <a name="viz"></a>

In [12]:
import open3d as o3d
import numpy as np

# black backgroud on 'k' key press
def change_background_to_black(vis):
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0, 0, 0])
    return False

key_to_callback = {}
key_to_callback[ord("K")] = change_background_to_black

# load scene from file
final_scene = o3d.io.read_point_cloud("monroe_final_scene.ply")

# creating an optional bounding box
aabb = final_scene.get_axis_aligned_bounding_box() 
aabb.color = (118/255, 118/255, 118/255)

# 1. skybox visualization
# o3d.visualization.draw([final_scene, aabb])

# 2. black background visualization
o3d.visualization.draw_geometries_with_key_callbacks([final_scene, aabb], key_to_callback)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


<hr>

## Example images <a name="images"></a>
## Raw data, no objects defined
<img src="Images/monroe_raw.png" width=900/>

## Monroe Scene creation with multiple objects
<img src="Images/monroe_colorized_scene.png" width=900/>

## Adding more objects to scene
<img src="Images/monroe_more_items.png" width=900/>

## Adding a second person
<img src="Images/monroe3.png" width=900/>

## Adding the sidewalk and beginning use of multi-color palettes
<img src="Images/monroe_sidewalk_and_colors.png" width=900/>

## Continuing multi-color palette implementation
<img src="Images/monroe4.png" width=900/>

## More coloring and objects
<img src="Images/monroe5.png" width=900/>

## Final scene loaded and visualized as single .ply file
<img src="Images/monroe_final.png" width=900/>
<img src="Images/monroe_google_3d.png" width=900/>
<img src="Images/monroe_street_view.png" width=900/>